In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import os
import re
import pandas as pd
import matplotlib.pylab as plt

from image_analysis.read_files import get_paths, change_paths
from image_analysis.fit import plot, roi, regionprops
from image_analysis.preprocess import mask

from tqdm import tqdm

In [3]:
pixel_size = 2

In [4]:
folder_pretreated = r'/home/natalia/Documents/github_image_analysis/experimental_data/treatment_all/'
df = pd.read_excel("../filtering_image/preview/description_of_all_images_exclude_not_nice_segm.xlsx")

In [5]:
# Remove rows where the 'exclude' column is 'yes'
df = df[df['exclude'] != 'yes']

df = df.drop(columns=['exclude'])
df = df.drop(columns=['comment'])
df = df.dropna(subset=['time'])
df['time'] = df['time'].astype(int)

# Reset the index
df.reset_index(drop=True, inplace=True)

In [6]:
df

,pretreated_file_name,hcr,animal_size,time,shh_segm,fgf_segm,dusp_segm
0,hcr11_lb27_fl,11,1.5,1,yes,yes,yes
1,hcr11_lb28_fl,11,1.5,1,yes,yes,yes
2,hcr11_lb28_fr,11,1.5,1,yes,yes,yes
3,hcr15_lb41_fl,15,1.5,1,yes,yes,yes
4,hcr16_lb44_fl,16,1.5,1,yes,yes,yes
...,...,...,...,...,...,...,...
58,hcr17_7cm_9dpa_dapi,17,7.0,9,yes,yes,yes
59,hcr21_7cm_10dpa_noif_17,21,7.0,10,yes,yes,yes
60,hcr21_7cm_10dpa_noif_16,21,7.0,10,yes,yes,no
61,hcr22_7cm_10d_t3a,22,7.0,10,yes,yes,yes


In [7]:
for pc in [40, 50, 60, 70, 80]:

    df[f'system_size_L_along_SF_pc{pc}_um']  = pd.NA
    df[f'length_L_SF_pc{pc}_um']  = pd.NA
    
    df[f'x0_pc{pc}_um_shh']  = pd.NA
    df[f'x0_pc{pc}_um_fgf'] = pd.NA

In [8]:
for i, file_name in tqdm( enumerate( df['pretreated_file_name'].values) ):
    
    file = os.path.join( folder_pretreated, file_name) + '.npy'
    
    keys = np.load( file.split('.npy')[0] + '_keys.npy' )
    image = dict(zip(
        keys,
        np.load( file )
    ))

    for pc in [40, 50, 60, 70, 80]:
        if (f'shh{pc}_clip' in keys) and (f'fgf{pc}_clip' in keys):
            
            image_to_rotate = dict(zip(
                ['shh_clip', 'fgf_clip', 'tissue_mask_with_epi'],
                [image[f'shh{pc}_clip'], image[f'fgf{pc}_clip'], image['tissue_mask_with_epi']]
            ))
             
            regionprops_3d = regionprops.regionprops_3d(image_to_rotate, 1, 1, 1, pixel_size)
            image_to_rotate = regionprops_3d.rotate_sf_to_ey()

            S = regionprops.center_mass( image_to_rotate['shh_clip'] )
            F = regionprops.center_mass( image_to_rotate['fgf_clip'] )

            L = np.sum( 
                mask.largest_component( image_to_rotate['tissue_mask_with_epi'][S[0], :, S[2]]  )
            ) * pixel_size
            L_SF  = np.abs( S[1] - F[1] ) * pixel_size
            
            ymin = np.nonzero( mask.largest_component(image_to_rotate['tissue_mask_with_epi'][S[0], :, S[2]]) )[0][0]
            ymax = np.nonzero( mask.largest_component(image_to_rotate['tissue_mask_with_epi'][S[0], :, S[2]]) )[0][-1]            
            
            x0_S = min( (S[1] - ymin) * pixel_size, (ymax - S[1]) * pixel_size )
            x0_F = min( (F[1] - ymin) * pixel_size, (ymax - F[1]) * pixel_size )

            df.loc[df['pretreated_file_name'] == file_name, f'system_size_L_along_SF_pc{pc}_um']  = L
            df.loc[df['pretreated_file_name'] == file_name, f'length_L_SF_pc{pc}_um']             = L_SF
            df.loc[df['pretreated_file_name'] == file_name, f'x0_pc{pc}_um_shh']                  = x0_S
            df.loc[df['pretreated_file_name'] == file_name, f'x0_pc{pc}_um_fgf']                  = x0_F
            
df.to_excel("system_size_L_and_gaps_x0/system_size_L_and_gaps_x0.xlsx", index=False)

0it [00:00, ?it/s]/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:25: RuntimeWarning: Mean of empty slice
  img_arr_x = np.nanmean(np.nanmean(img_arr, axis=0), axis=0)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:28: RuntimeWarning: Mean of empty slice
  img_arr_y = np.nanmean(np.nanmean(img_arr, axis=0), axis=1)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:31: RuntimeWarning: Mean of empty slice
  img_arr_z = np.nanmean(np.nanmean(img_arr, axis=1), axis=1)


rotation around x:
-66.8014094863518 0 0
dictionary, with 3d data
dictionary, with 3d data


/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:25: RuntimeWarning: Mean of empty slice
  img_arr_x = np.nanmean(np.nanmean(img_arr, axis=0), axis=0)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:28: RuntimeWarning: Mean of empty slice
  img_arr_y = np.nanmean(np.nanmean(img_arr, axis=0), axis=1)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:31: RuntimeWarning: Mean of empty slice
  img_arr_z = np.nanmean(np.nanmean(img_arr, axis=1), axis=1)


rotation around y:
0 0 54.04177572930654
dictionary, with 3d data
dictionary, with 3d data
angle beetween ey and sf: 3


/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:25: RuntimeWarning: Mean of empty slice
  img_arr_x = np.nanmean(np.nanmean(img_arr, axis=0), axis=0)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:28: RuntimeWarning: Mean of empty slice
  img_arr_y = np.nanmean(np.nanmean(img_arr, axis=0), axis=1)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:31: RuntimeWarning: Mean of empty slice
  img_arr_z = np.nanmean(np.nanmean(img_arr, axis=1), axis=1)


...

/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:25: RuntimeWarning: Mean of empty slice
  img_arr_x = np.nanmean(np.nanmean(img_arr, axis=0), axis=0)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:28: RuntimeWarning: Mean of empty slice
  img_arr_y = np.nanmean(np.nanmean(img_arr, axis=0), axis=1)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:31: RuntimeWarning: Mean of empty slice
  img_arr_z = np.nanmean(np.nanmean(img_arr, axis=1), axis=1)


rotation around x:
-34.200484130268535 0 0
dictionary, with 3d data
dictionary, with 3d data
rotation around y:
0 0 62.718701973426136
dictionary, with 3d data
dictionary, with 3d data
angle beetween ey and sf: 3


/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:25: RuntimeWarning: Mean of empty slice
  img_arr_x = np.nanmean(np.nanmean(img_arr, axis=0), axis=0)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:28: RuntimeWarning: Mean of empty slice
  img_arr_y = np.nanmean(np.nanmean(img_arr, axis=0), axis=1)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:31: RuntimeWarning: Mean of empty slice
  img_arr_z = np.nanmean(np.nanmean(img_arr, axis=1), axis=1)


rotation around x:
-34.665497333306206 0 0
dictionary, with 3d data
dictionary, with 3d data
rotation around y:
0 0 60.84612039566949
dictionary, with 3d data
dictionary, with 3d data
angle beetween ey and sf: -5


63it [26:58, 25.69s/it]
